In [4]:
# ROTAS (trechos)
ROTAS: List[Dict[str, str]] = [
    {"origem_url": "vitoria-da-conquista-ba-todos", "destino_url": "sao-paulo-sp-todos"},
    {"origem_url": "sao-paulo-sp-todos", "destino_url": "vitoria-da-conquista-ba-todos"},
]

# =========================
# SELETORES
# =========================
SEL_DEPARTURE_TIME = "time.departure-time, time[data-testid='departure-time']"
SEL_COMPANY        = "div.company, div[data-testid='company-name']"
SEL_CLASS          = "div.service-class, span[data-testid='service-class']"
SEL_PRICE          = "span.price-value, span[data-testid='trip-card-price']"
SEL_RESULTS_CONTAINER = "div.search-results, div[data-testid='search-results']"

# =========================
# UTILITÁRIAS
# =========================
def build_driver() -> webdriver.Chrome:
    opts = Options()
    if HEADLESS:
        opts.add_argument("--headless=new")
    opts.add_argument("--window-size=1366,768")
    opts.add_argument("--disable-gpu")
    opts.add_argument("--no-sandbox")
    opts.add_argument("--disable-dev-shm-usage")
    opts.add_argument("--lang=pt-BR")
    drv = webdriver.Chrome(options=opts)
    drv.set_page_load_timeout(60)
    return drv

def wait_for_results(driver):
    WebDriverWait(driver, TIMEOUT).until(
        EC.any_of(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, SEL_DEPARTURE_TIME)),
            EC.presence_of_element_located((By.CSS_SELECTOR, SEL_RESULTS_CONTAINER)),
        )
    )

def infinite_scroll(driver, max_scrolls=MAX_SCROLLS, pause=PAUSA_SCROLL):
    last_height = driver.execute_script("return document.body.scrollHeight")
    for _ in range(max_scrolls):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(pause)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

def safe_text(el) -> str:
    try:
        return el.text.strip()
    except Exception:
        return ""

def open_page(driver, origem_url, destino_url, data_str):
    url = f"https://www.clickbus.com.br/onibus/{origem_url}/{destino_url}?departureDate={data_str}"
    print(f"Acessando: {url}")
    driver.get(url)

# =========================
# COLETA DE UM DIA/ROTA (preço já como float)
# =========================
def coletar_um_dia(driver, origem_url, destino_url, data_viagem: datetime):
    data_str = data_viagem.strftime("%Y-%m-%d")
    open_page(driver, origem_url, destino_url, data_str)
    wait_for_results(driver)
    time.sleep(2)
    infinite_scroll(driver)

    empresas = driver.find_elements(By.CSS_SELECTOR, SEL_COMPANY)
    horarios = driver.find_elements(By.CSS_SELECTOR, SEL_DEPARTURE_TIME)
    classes  = driver.find_elements(By.CSS_SELECTOR, SEL_CLASS)
    precos   = driver.find_elements(By.CSS_SELECTOR, SEL_PRICE)

    total = min(len(empresas), len(horarios), len(precos))
    print(f"→ {data_str} | {origem_url} → {destino_url} | Viagens: {total}")

    linhas = []
    for i in range(total):
        empresa = empresas[i].get_attribute("data-content") or safe_text(empresas[i])
        saida   = safe_text(horarios[i])
        classe  = safe_text(classes[i]) if i < len(classes) else ""
        preco_str = safe_text(precos[i]).replace("R$", "").replace("\xa0", " ").strip()
        # --- conversão BR -> float ---
        try:
            preco = float(preco_str.replace(".", "").replace(",", "."))
        except ValueError:
            preco = None

        linhas.append({
            "data_viagem": data_str,
            "empresa": empresa,
            "horario_saida": saida,
            "classe": classe,
            "preco": preco,   # já float
        })
    return linhas

# =========================
# COLETA POR TRECHO (RETORNA UM DF POR ROTA)
# =========================
def coletar_por_trecho(rotas: List[Dict[str, str]], dias: int = DIAS_A_COLETAR) -> Dict[str, pd.DataFrame]:
    """
    Retorna um dicionário {nome_trecho: DataFrame}, um DF por rota.
    Colunas: data_viagem, empresa, horario_saida, classe, preco (float).
    """
    if dias < 1:
        dias = 1
    hoje = datetime.now().date()

    dfs_por_trecho: Dict[str, pd.DataFrame] = {}

    for idx, rota in enumerate(rotas, start=1):
        origem_url  = rota["origem_url"]
        destino_url = rota["destino_url"]
        nome_trecho = f"{origem_url}__{destino_url}"  # chave do dict (mantenha ou personalize)

        print("=" * 80)
        print(f"[{idx}/{len(rotas)}] Trecho: {origem_url} -> {destino_url}")

        driver = build_driver()
        linhas_trecho = []
        try:
            for i in range(dias):  # se dias=1 -> só hoje
                dia = hoje + timedelta(days=i)
                print("-" * 80)
                print(f"Dia {i+1}/{dias} | {dia}")
                try:
                    linhas = coletar_um_dia(driver, origem_url, destino_url, dia)
                    linhas_trecho.extend(linhas)
                except Exception as e:
                    print(f"[ERRO] {dia} {origem_url}->{destino_url}: {e}")
                    try: driver.quit()
                    except: pass
                    driver = build_driver()
                time.sleep(2)
        finally:
            try: driver.quit()
            except: pass

        df_trecho = pd.DataFrame(linhas_trecho)

        if not df_trecho.empty:
            df_trecho["data_viagem"] = pd.to_datetime(df_trecho["data_viagem"], errors="coerce")
            # já está float; ordena
            df_trecho = df_trecho.drop_duplicates().sort_values(["data_viagem", "horario_saida"], na_position="last")

        dfs_por_trecho[nome_trecho] = df_trecho

    return dfs_por_trecho

# =========================
# EXECUÇÃO
# =========================
dfs = coletar_por_trecho(ROTAS, dias=DIAS_A_COLETAR)

# Preview: um pedacinho de cada DF
for nome, df in dfs.items():
    print("\n===== TRECHO:", nome, "=====")
    print(df)

[1/2] Trecho: vitoria-da-conquista-ba-todos -> sao-paulo-sp-todos
--------------------------------------------------------------------------------
Dia 1/3 | 2025-08-16
Acessando: https://www.clickbus.com.br/onibus/vitoria-da-conquista-ba-todos/sao-paulo-sp-todos?departureDate=2025-08-16
→ 2025-08-16 | vitoria-da-conquista-ba-todos → sao-paulo-sp-todos | Viagens: 21
--------------------------------------------------------------------------------
Dia 2/3 | 2025-08-17
Acessando: https://www.clickbus.com.br/onibus/vitoria-da-conquista-ba-todos/sao-paulo-sp-todos?departureDate=2025-08-17
→ 2025-08-17 | vitoria-da-conquista-ba-todos → sao-paulo-sp-todos | Viagens: 18
--------------------------------------------------------------------------------
Dia 3/3 | 2025-08-18
Acessando: https://www.clickbus.com.br/onibus/vitoria-da-conquista-ba-todos/sao-paulo-sp-todos?departureDate=2025-08-18
→ 2025-08-18 | vitoria-da-conquista-ba-todos → sao-paulo-sp-todos | Viagens: 16
[2/2] Trecho: sao-paulo-sp-t

In [7]:
# Exporta todos os dataframes para um único arquivo Excel
nome_arquivo = "passagens_clickbus_rotas.xlsx"

with pd.ExcelWriter(nome_arquivo, engine="openpyxl") as writer:
    for nome_trecho, df in dfs.items():
        sheet_name = nome_trecho[:31]
        df.to_excel(writer, sheet_name=sheet_name, index=False)